In [ ]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
from datetime import datetime, date
from psx import stocks, tickers
from IPython.display import display
import pandas as pd
import investpy
import yfinance as yf
from matplotlib.dates import relativedelta
from apps.environment.models import Stock
from apps.environment.models.stock import StockBuffer
from apps.environment.models.commodity import Commodity
from apps.environment.models.commodity import CommodityBuffer
import pytz
from os import environ
from math import ceil, floor
import psycopg2



In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
# Assuming tickers is your DataFrame


tickers_column = tickers()
# display(tickers_column)
symbols = list(tickers_column["symbol"])
names = list(tickers_column["name"])
len_syms = len(symbols)
# # print(symbols)
hundred_inds = pd.read_csv('KSE100 indx Comps - stock-exchange-kse-100pakistan.csv').to_dict(orient='records')
# print(hundred_inds)
hun_syms = {}

all_syms = {}
for ind in range(len_syms):
    all_syms[symbols[ind]] = names[ind]

the_index = 1
for hun_ind in hundred_inds:
    if hun_ind["SYMBOL"] in all_syms:
        sym_b = hun_ind["SYMBOL"]
        if sym_b not in hun_syms:
            hun_syms[sym_b] = {
                "index": the_index,
                "name": all_syms[sym_b]
            }
            the_index = the_index + 1

db_stcks = {}
all_stcks = Stock.objects.all()
max_indx = len(all_stcks)
for astck in all_stcks:
    db_stcks[astck.symbol] = astck.symbol

new_stcks_list = []
for sym_bl in hun_syms:
    if sym_bl not in db_stcks:
        sym_data = hun_syms[sym_bl]
        new_indx = sym_data["index"]
        if new_indx<=max_indx:
            new_indx = max_indx+1
        max_indx = max_indx+1
        new_stcks_list.append(
            Stock(
                index=sym_data["index"],
                symbol=sym_bl,
                name=sym_data["name"],
                type=100
            )
        )

Stock.objects.bulk_create(new_stcks_list)

In [ ]:


all_stocks = Stock.objects.all()

stock_syms = {}

for a_stock in all_stocks:
    if a_stock.symbol not in stock_syms:
        stock_syms[a_stock.symbol] = {
            "object": a_stock,
            "snapshots": {},
            "last_date": None
        }

snapshots = StockBuffer.objects.select_related("stock").all()
for snapshot in snapshots:
    snp_stck = snapshot.stock
    snp_stck_symbl = snp_stck.symbol
    if snp_stck_symbl in stock_syms:
        snp_data = stock_syms[snp_stck_symbl]
        stck_snps = snp_data["snapshots"]
        if snapshot.captured_at not in stck_snps:
            stck_snps[snapshot.captured_at.strftime('%Y-%m-%d %H:%M:%S')] = snapshot
            if not snp_data["last_date"]:
                snp_data["last_date"] = snapshot.captured_at.date()

# tst = 1
for stck_sym, stck_data in stock_syms.items():
    new_snapshots = []
    last_date: datetime = stck_data["last_date"]
    if last_date is None:
        start_from = date(2020, 1, 1)
    else:
        start_from = last_date
    print(last_date, stck_sym, start_from)
    end_date = pytz.utc.localize(datetime.now() + relativedelta(days=2)).date()
    data = stocks(stck_sym, start=start_from, end=end_date)
    data_points_len = len(data)
    stck_snpshts = stck_data["snapshots"]
    the_stock: Stock = stck_data["object"]
    last_data_point = None
    for digi in range(data_points_len):
        data_point = data.iloc[digi]
        time_point = data_point.name.to_pydatetime()
        time_point_str = data_point.name.strftime('%Y-%m-%d %H:%M:%S')
        if time_point_str not in stck_snpshts:
            zone_point_time = pytz.utc.localize(time_point)
            nw_price = data_point.Close
            if last_data_point is None:
                change = 0
            else:
                old_price = last_data_point.Close
                if old_price and old_price > 0:
                    change = (nw_price-old_price)/old_price
                else:
                    change = 0
            new_snapshots.append(StockBuffer(
                stock = the_stock,
                captured_at = zone_point_time,
                price_snapshot = nw_price,
                change = change,
                volume = data_point.Volume,
                bid_vol = 100000000,
                bid_price = nw_price,
                offer_vol = 100000000,
                offer_price = nw_price,
                open = data_point.Open,
                close = nw_price,
                high = data_point.High,
                low = data_point.Low
            ))
        last_data_point = data_point
    StockBuffer.objects.bulk_create(new_snapshots)

    # tst = tst + 1
    # if tst == 4:
    #     break

In [ ]:


all_commodities = Commodity.objects.all()

commodity_syms = {}

for a_commodity in all_commodities:
    if a_commodity.symbol not in commodity_syms:
        commodity_syms[a_commodity.symbol] = {
            "object": a_commodity,
            "snapshots": {},
            "last_date": None
        }

snapshots = CommodityBuffer.objects.select_related("commodity").all()
for snapshot in snapshots:
    snp_cmmdty = snapshot.commodity
    snp_cmmdty_symbl = snp_cmmdty.symbol
    if snp_cmmdty_symbl in commodity_syms:
        cmmdty_data = commodity_syms[snp_cmmdty_symbl]
        cmmdty_snps = cmmdty_data["snapshots"]
        if snapshot.captured_at not in cmmdty_snps:
            cmmdty_snps[snapshot.captured_at.strftime('%Y-%m-%d %H:%M:%S')] = snapshot
            if not cmmdty_data["last_date"]:
                cmmdty_data["last_date"] = snapshot.captured_at.date()

# tst = 1
for cmmdty_sym, cmmdty_data in commodity_syms.items():
    new_snapshots = []
    last_date: datetime = cmmdty_data["last_date"]
    if last_date is None:
        start_from = date(2019, 12, 30)
    else:
        start_from = date(2019, 12, 30)
        # start_from = last_date
    print(last_date, cmmdty_sym, start_from)
    end_date = pytz.utc.localize(datetime.now() + relativedelta(days=2)).date()
    data = yf.download(cmmdty_sym, start=start_from, end=end_date)
    data_points_len = len(data)
    cmmdty_snpshts = cmmdty_data["snapshots"]
    the_commodity: Commodity = cmmdty_data["object"]
    last_data_point = None
    for digi in range(data_points_len):
        data_point = data.iloc[digi]
        time_point = data_point.name.to_pydatetime()
        time_point_str = data_point.name.strftime('%Y-%m-%d %H:%M:%S')
        if time_point_str not in cmmdty_snpshts:
            zone_point_time = pytz.utc.localize(time_point)
            nw_price = data_point.Close
            if last_data_point is None:
                change = 0
            else:
                old_price = last_data_point.Close
                if old_price and old_price > 0:
                    change = (nw_price-old_price)/old_price
                else:
                    change = 0
            new_snapshots.append(CommodityBuffer(
                commodity = the_commodity,
                captured_at = zone_point_time,
                price_snapshot = nw_price,
            ))
        last_data_point = data_point
    CommodityBuffer.objects.bulk_create(new_snapshots)

    # tst = tst + 1
    # if tst == 4:
    #     break

In [ ]:
starting_time_step = datetime(year=2020, month=1, day=1)

db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
env_config = {
    "db_params": db_params, 
    "max_episode_steps": 1000, 
    "the_current_time_step": starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 200,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "preparing": True
} 

In [ ]:
the_current_time_step = env_config.get("the_current_time_step")
max_epi_len = 50

__last_time_step = the_current_time_step + relativedelta(days=max_epi_len)
the_current_time_step = pytz.utc.localize(datetime.strptime(str(the_current_time_step), '%Y-%m-%d %H:%M:%S'))
__last_time_step = pytz.utc.localize(datetime.strptime(str(__last_time_step), '%Y-%m-%d %H:%M:%S'))
str_time_step = str(the_current_time_step)

str_last_time_step = str(__last_time_step)
db_params = env_config.get("db_params")
db_conn = psycopg2.connect(**db_params)
cursor = db_conn.cursor()

stcks_query = """
    SELECT *
    FROM stocks
"""
cursor.execute(stcks_query)
stck_data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
stck_df = pd.DataFrame(stck_data, columns=column_names) 
# stck_df = stck_df.to_dict(orient='records')

stcks_buffer_query = f"""
    SELECT stocks_buffers.*, stocks.index
    FROM stocks_buffers 
    JOIN stocks on stocks.id = stocks_buffers.stock_id
    WHERE 
    captured_at >= '{str_time_step}' AND captured_at <= '{str_last_time_step}'
"""
cursor.execute(stcks_buffer_query)
stcks_buffer_data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
stcks_buffer_df = pd.DataFrame(stcks_buffer_data, columns=column_names)

# stcks_buffer_df = stcks_buffer_df.to_dict(orient='records')

cmmdties_buffer_query = f"""
    SELECT commodities_buffers.*, commodities.index
    FROM commodities_buffers 
    JOIN commodities on commodities.id = commodities_buffers.commodity_id
    WHERE 
    captured_at >= '{str_time_step}' AND captured_at <= '{str_last_time_step}'
"""
cursor.execute(cmmdties_buffer_query)
cmmdties_buffer_data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
cmmdties_buffer_df = pd.DataFrame(cmmdties_buffer_data, columns=column_names)
# cmmdties_buffer_df = cmmdties_buffer_df.to_dict(orient='records')

cmmdties_query = """
    SELECT *
    FROM commodities
"""
cursor.execute(cmmdties_query)
cmmdties_data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
cmmdties_df = pd.DataFrame(cmmdties_data, columns=column_names) 

In [65]:
# Your query and execution
stcks_buffer_query = """
    SELECT stocks_buffers.*, stocks.index
    FROM stocks_buffers 
    JOIN stocks on stocks.id = stocks_buffers.stock_id
    WHERE 
    captured_at >= %s AND captured_at <= %s
"""
cursor.execute(stcks_buffer_query, [str_time_step, str_last_time_step])
stcks_buffer_data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
stcks_buffer_df = pd.DataFrame(stcks_buffer_data, columns=column_names)

# Ensure that the 'captured_at' column is in datetime format
stcks_buffer_df['captured_at'] = pd.to_datetime(stcks_buffer_df['captured_at'])

# Use pivot to create a row for each date
pivot_df = stcks_buffer_df.pivot(index='captured_at', columns='index', values=column_names)

# Reset index to make 'captured_at' a regular column
pivot_df.reset_index(inplace=True)

# If needed, you can fill missing values with the previous row's values
pivot_df.fillna(method='ffill', inplace=True)

# print(pivot_df)
specific_datetime = pd.to_datetime('2020-01-01 00:00:00+00:00')

# Filter the DataFrame for the specified datetime
specific_data = pivot_df[pivot_df['captured_at'] == specific_datetime]

display(specific_data)

InterfaceError: cursor already closed